GROUP 36 PROJECT PROPOSAL

Analyzing the Effect of Different Attributes on Heart Health: Using Cleveland Heart Disease Data

1. Research Question

What is the probablity of a person getting heart disease?

2. Introduction

As of 2018, 30.3 million U.S. adults are diagnosed with heart disease. Heart disease is a group of conditions that affect the structure or function of the heart. The four most common types of heart disease are coronary artery disease, heart valve disease, heart failure, and arrhythmia. This project aims to predict a person's likelihood of getting heart disease using the attributes, age, sex, cholesterol level, blood sugar level, chest pain, and resting blood pressure. The project will produce a predictor that has been trained with multiple folds of data so that it can accurately predict a person’s likelihood of getting heart disease. The data set given to us is from the Cleveland database primarily focuses on 14 attributes: age, sex, chest pain type, resting blood pressure, cholesterol, fasting blood sugar, resting electrocardiographic results, maximum heart rate achieved, exercise-induced angina, ST depression induced by exercise relative to rest,  number of major vessels, thal, and diagnosis of heart disease.

3. Preliminary Exploratory Data Analysis

Import Libraries

In [1]:
library(tidyverse)
library(tidymodels)
library(RColorBrewer)
library(ggplot2)
library(digest)
library(repr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

Preview Data

In [2]:
set.seed(1234) 
dir.create("data_for_project")

url <- "https://raw.githubusercontent.com/slakhwar/grp-project/main/processed.cleveland.csv"

download.file(url, destfile ="data_for_project/processed.cleveland.csv")
cleveland_data <- read.csv("data_for_project/processed.cleveland.csv")

# cleveland_p_data <- cleveland_data %>%
#  select(age, sex, cp..chest.pain.type., trestbps..resting.blood.pressure., chol..serum.cholestoral.in.mg.dl., fbs..fasting.blood.sugar...120.mg.dl.,num..diagnosis.of.heart.disease..angiographic.disease.status..)
# rename columns
colnames(cleveland_data) <- c("Age", "Sex", "cp_type", "rest_bp", "serum_chol", "fast_bs", "rest_ecr", "max_hr", "ex_ia", "ex_dep_rest", "sl_pe", "num_mv_cf", "thal", "diag_hd_angio_ds")
head(cleveland_data, 10)

Warning message in dir.create("data_for_project"):
“'data_for_project' already exists”


,Age,Sex,cp_type,rest_bp,serum_chol,fast_bs,rest_ecr,max_hr,ex_ia,ex_dep_rest,sl_pe,num_mv_cf,thal,diag_hd_angio_ds
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<chr>,<chr>,<int>
1,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0
2,67,1,4,160,286,0,2,108,1,1.5,2,3,3,2
3,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1
4,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
5,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0
6,56,1,2,120,236,0,0,178,0,0.8,1,0,3,0
7,62,0,4,140,268,0,2,160,0,3.6,3,2,3,3
8,57,0,4,120,354,0,0,163,1,0.6,1,0,3,0
9,63,1,4,130,254,0,2,147,0,1.4,2,1,7,2


Summary Statistics Table

In [5]:
cleveland_p_data <- cleveland_data %>%
    select(Age, Sex, cp_type, rest_bp, serum_chol, fast_bs, diag_hd_angio_ds) 
cleveland_p_data

cleveland_p_data_split <- initial_split(cleveland_p_data, prop=0.75, strata = diag_hd_angio_ds)
cleveland_p_data_train <- training(cleveland_p_data_split)
cleveland_p_data_test <- testing(cleveland_p_data_split)

total_rows <- cleveland_p_data_train %>%
    count() %>%
    pull()

#Calculating percentage chance of getting heart disease by gender
gender_percent_table <- cleveland_p_data_train %>%
    mutate(Sex = ifelse(Sex>0, "Male", "Female")) %>%
    group_by(Sex) %>%
    summarise(percent = n()*100/total_rows)
gender_percent_table

knitr::kable(gender_percent_table, caption = "Figurex")

#Mean for all predictors

mean_hd_diag_table <- cleveland_p_data_train %>%
        map_dfc(mean) %>%
        pivot_longer(
        cols = Age : diag_hd_angio_ds,
        names_to = "Variable",
        values_to = "Mean")
# Standard Deviation for all predictors
sd_hd_diag_table <- cleveland_p_data_train %>%
        map_dfc(sd) %>%
        pivot_longer(
        cols = Age : diag_hd_angio_ds,
        names_to = "Variable",
        values_to = "Standard Deviation") %>%
        select(-Variable)

#Grouping Mean and Standard Deviation in the same table
mean_sd_table <- mean_hd_diag_table %>%
            bind_cols(sd_hd_diag_table)

knitr::kable(mean_sd_table, caption ="Figure_y")

asymptomatic_cp_hd <- cleveland_p_data_train %>%
    select(Age, Sex, cp_type, rest_bp, serum_chol, fast_bs, diag_hd_angio_ds) 
    head(asymptomatic_cp_hd, 10)

Age,Sex,cp_type,rest_bp,serum_chol,fast_bs,diag_hd_angio_ds
<int>,<int>,<int>,<int>,<int>,<int>,<int>
63,1,1,145,233,1,0
67,1,4,160,286,0,2
67,1,4,120,229,0,1
37,1,3,130,250,0,0
41,0,2,130,204,0,0
56,1,2,120,236,0,0
62,0,4,140,268,0,3
57,0,4,120,354,0,0
63,1,4,130,254,0,2


`summarise()` ungrouping output (override with `.groups` argument)



Sex,percent
<chr>,<dbl>
Female,31.57895
Male,68.42105




Table: Figurex

|Sex    |  percent|
|:------|--------:|
|Female | 31.57895|
|Male   | 68.42105|



Table: Figure_y

|Variable         |        Mean| Standard Deviation|
|:----------------|-----------:|------------------:|
|Age              |  54.3026316|          8.8978833|
|Sex              |   0.6842105|          0.4658522|
|cp_type          |   3.2017544|          0.9588891|
|rest_bp          | 131.0657895|         17.6238241|
|serum_chol       | 246.2368421|         51.6051260|
|fast_bs          |   0.1447368|          0.3526094|
|diag_hd_angio_ds |   0.9342105|          1.2310469|

,Age,Sex,cp_type,rest_bp,serum_chol,fast_bs,diag_hd_angio_ds
,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,63,1,1,145,233,1,0
2,67,1,4,160,286,0,2
3,67,1,4,120,229,0,1
4,37,1,3,130,250,0,0
6,56,1,2,120,236,0,0
7,62,0,4,140,268,0,3
9,63,1,4,130,254,0,2
11,57,1,4,140,192,0,0
12,56,0,2,140,294,0,0


In [10]:

### BEGIN SOLUTION
hd_knn_recipe <- recipe(diag_hd_angio_ds ~., data = cleveland_p_data_train)
hd_knn_recipe

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
            set_engine("kknn") %>%
            set_mode("classification")

hd_knn_vfold <- vfold_cv(cleveland_p_data_train, v = 5, strata = diag_hd_angio_ds)


hd_knn_results <- workflow() %>%
                  add_recipe(hd_knn_recipe) %>%
                  add_model(knn_spec) %>%
                  tune_grid(resamples = hd_knn_vfold, grid = 10) %>%
                  collect_metrics()

accuracies <- hd_knn_results %>%
                 filter(.metric == 'accuracy')
accuracies

Data Recipe

Inputs:

      role #variables
   outcome          1
 predictor          6

x Fold1: model 1/1: Error: For classification models, the outcome should be a fac...

x Fold2: model 1/1: Error: For classification models, the outcome should be a fac...

x Fold3: model 1/1: Error: For classification models, the outcome should be a fac...

x Fold4: model 1/1: Error: For classification models, the outcome should be a fac...

x Fold5: model 1/1: Error: For classification models, the outcome should be a fac...

Warning message:
“All models failed in tune_grid(). See the `.notes` column.”


ERROR: Error: All of the models failed. See the .notes column.


Visualization

In [12]:
# accuracy graph

accuracy_hd <- ggplot(accuracies, aes(x = neighbors , y = mean)) +
    geom_point() +
    geom_line() +
    labs(x = "Neighbors", y = "Accuracy Estimate") +
    theme(text = element_text(size = 12))
accuracy_hd

hd_plot <- ggplot(cleveland_p_data_train, aes(x=age, fill=as_factor(sex))) +
 geom_histogram(binwidth=1)+
 labs(x="Age", y="Frequency of Heart Disease", title="Distribution of Heart Disease by sex") +
 scale_fill_discrete(name = "Gender", labels = c("0(Female)","1(Male)"))


hd_plot


hd_cp_plot <- ggplot(cleveland_p_data_train, aes(x=num..diagnosis.of.heart.disease..angiographic.disease.status.., fill=as_factor(cp..chest.pain.type.))) +
 geom_histogram(binwidth=1) +
 labs(x="Stages of Heart Disease(angiographic disease status)", y="Frequency of Heart Disease", title="Distribution of Stages of Heart Disease by Types of Chest Pains")+
 scale_fill_discrete(name = "Types of Chest Pains", labels = c("1(typical angina)","2(atypical angina)","3(non-anginal pain)","4(asymptomatic)"))


hd_cp_plot

ERROR: Error in ggplot(accuracies, aes(x = neighbors, y = mean)): object 'accuracies' not found


Create a new model spec with the best K

In [ ]:
# pick the best K (highest accuracy and does not change much if you change k to nearvy value)

# make a new model spec with the best K, retrain classifier with fit, then use predict() function

hd_knn_recipe <- recipe(diag_hd_angio_ds ~., data = cleveland_p_data_train) %>%
    step_scale(all_predictors()) %>%
    step_center(all_predictors())
hd_knn_recipe

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = # whatever the best k was) %>%
    set_engine("kknn") %>%
    set_model("classification")

knn_fit <- workflow() %>%
    add_recipe(hd_knn_recipe) %>%
    add_model(knn_spec) %>%
fit(data = cleveland_p_data_train)
                             
hd_test_predictions <- predict(knn_fit, cleveland_p_data_test) %>%
    bind_cols(cleveland_p_data_test) %>%
    metrics(truth = diag_hd_angio_ds, estimate = .pred_class) %>%
    filter(.metric == "accuracy") %>%
    select(mean) %>%
    pull()
hd_test_predictions

confusion <- hd_test_predictions %>%
    conf_mat(truth = diag_hd_angio_ds, estimate = .pred_class)
confusion

4. Methods

We will be using the data on age, sex, cholesterol, diagnosis of heart disease, resting blood pressure and chest pain type categories from our data set. 

In our project, we will use a linear regression and will be directly comparing mainly the categories of age and cholesterol. From there, we will further analyze the data by also taking into account the diagnosis of different heart diseases and how common the different chest pain types are in the different age groups and how they differ by sex.

Since we are using multiple different categories, we will use a few different graphs.

One way we will visualize our data is through bar graphs. One example of a bar graph we will use is one comparing frequency of heart disease (y) and age (x). This graph will also be separated into gender through color in the bar graph (male blue and females pink). Since we are also comparing types of chest pain in our analysis, we will also use a bar graph to analyze the frequency of the different chest pains at different stages of heart disease. 

We will visualize the results through a scatter plot. Our scatter plot  will compare Age (x axis) and cholesterol levels (y axis). We will also use colour and a legend to display gender (ex- male blue and female in pink). 
We will then break the graph down further into two graphs, one with just female data and one with just male data. This will allow us to see trends in all of the categories we use and then also compare them visually with ease. 


5. Expected Outcomes and Significance

We suspect people with a combination of age and blood sugar level have a higher chance of getting heart disease.

These findings could encourage people to watch their health closely by eating more healthily and exercising. It could also warn people with one or more of the attributes listed above to get a medical checkup.

  What future questions could this lead to?

How do we regulate what food is made out of to minimize blood sugar levels? And if age is most correlated to heart disease, does the food that person ate in their generation affect heart disease frequency? Does our food safety policy need to be reworked for the betterment of the health of the future generation? 